In [40]:
import neptune
from neptune.sessions import Session
from pathlib import Path
import json
import zipfile
from tqdm import tqdm

PROJECT = "kjang0517/msmarco"

session = Session()
project = session.get_project(project_qualified_name=PROJECT)
# neptune.init(project_qualified_name="kjang0517/msmarco")
# neptune.get_experiments()

# 실험 기록 및 결과물 다운로드
* `state == 'succeeded'`
* `download_artifacts()`

In [54]:
channel_df

,x,val_loss_recover,epoch,val_loss_task,train_loss_recover,train_loss,train_loss_task,val_loss
0,0.0,NaN,0.0,NaN,0.027138,0.027138,0.0,NaN
1,50.0,NaN,0.0,NaN,0.024790,0.024790,0.0,NaN
2,100.0,NaN,0.0,NaN,0.021247,0.021247,0.0,NaN
3,150.0,NaN,0.0,NaN,0.018054,0.018054,0.0,NaN
4,200.0,NaN,0.0,NaN,0.015656,0.015656,0.0,NaN
...,...,...,...,...,...,...,...,...
595,26418.0,NaN,49.0,NaN,0.001109,0.001109,0.0,NaN
596,26468.0,NaN,49.0,NaN,0.001098,0.001098,0.0,NaN
597,26518.0,NaN,49.0,NaN,0.001105,0.001105,0.0,NaN
598,26568.0,NaN,49.0,NaN,0.001095,0.001095,0.0,NaN


In [ ]:
base_dir = Path("experiments")
if not base_dir.exists():
    base_dir.mkdir(parents=True)

target_exps = [exp for exp in project.get_experiments() if exp.state == "succeeded"]
for exp in tqdm(target_exps):
    destination_dir = base_dir / exp.id
    # check already exist
    if destination_dir.exists():
        print("Skipping", exp.id)
        continue
    else:
        destination_dir.mkdir(parents=True)

    # start fetching
    print("Fetching", exp.id)

    # parameters
    params = exp.get_parameters()
    with (destination_dir / "parameters.json").open("w", encoding="utf-8") as f:
        json.dump(params, f, indent=4)

    # channels
    channel_names = list(exp.get_channels().keys())
    channel_df = exp.get_numeric_channels_values(*channel_names).fillna(0.0)
    channel_df.to_csv(destination_dir / "channels.csv", index=False)

    # artifacts
    exp.download_artifacts(destination_dir=str(destination_dir))

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

Fetching MSMAR-103


  4%|███▎                                                                               | 1/25 [00:03<01:34,  3.93s/it]

Fetching MSMAR-106


  8%|██████▋                                                                            | 2/25 [00:07<01:29,  3.90s/it]

...

Fetching MSMAR-170


 48%|███████████████████████████████████████▎                                          | 12/25 [12:10<15:07, 69.79s/it]

Fetching MSMAR-176


 52%|██████████████████████████████████████████▋                                       | 13/25 [12:26<10:42, 53.58s/it]

Fetching MSMAR-177


In [ ]:
zip_files = list(base_dir.glob("**/*.zip"))
for zpath in zip_files:
    print("Processing", zpath)
    with zipfile.ZipFile(zpath) as zf:
        zf.extractall(path=zpath.parent)
    zpath.unlink()